## genTestCasesTool

### Install package

In [ ]:
# %pip install -U langchain-google-genai
# %pip install -U google-genai
# %pip install --upgrade --quiet  langchain-google-genai
# %pip install dotenv
# %pip install langchain-ollama
# %pip install --upgrade --quiet  langchain-ollama

# !pip install langchain-community
# !pip install unstructured
# !pip install pdfminer
# !pip install "unstructured[all-docs]" 

## Connect Ollama Local model

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI 
#from langchain_ollama import ChatOllama
from dotenv import load_dotenv

load= load_dotenv('../.env')
 

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


tc_llm = ChatGoogleGenerativeAI( 
   model="gemini-2.0-flash"
)

## Create tools
### Tools for Parallel Execution 

In [3]:
#import os
from langchain.tools import Tool
from langchain.prompts import PromptTemplate

#@tool
def generate_test_cases(user_story: str) -> str:
    """Generate test cases from a user stories."""
    prompt_template = PromptTemplate.from_template(
        """
        You are a QA Automation Engineer. 
        Your task is to convert the following user story into at least 10 test cases.in Gherkin BDD style  format.
        Include combinations of valid, invalid, and edge cases & alternative scenarios.

        User Story: {user_story}    

        Format:
        Feature: 
        Scenario:
            Given 
            When 
            Then
        And 



        """
                          

    )
    # ✅ Correct way to pass input
    prompt = prompt_template.invoke({"user_story": user_story})
    #return llm.invoke(prompt) 
    return tc_llm.invoke(prompt)


## initialize_agent

In [4]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool

# Wrap your function as a LangChain tool
@tool
def generate_test_cases(user_story: str) -> str:
    """Generate BDD test cases from a user story."""
    return f"Generated BDD test cases for: {user_story}"  # replace with real logic

# Initialize agent with proper tool
agent = initialize_agent(
    tools=[generate_test_cases],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Example run
user_story_input = """ 
As a user, I want to reset my password using a link sent to my email,
so that I can regain access to my account if I forgot my password.
"""
result = agent.invoke({"input": f"Create BDD Test Cases For {user_story_input}"})
print(result)


/tmp/ipykernel_114883/3052775936.py:11: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...
Action: generate_test_cases
Action Input: As a user, I want to reset my password using a link sent to my email, so that I can regain access to my account if I forgot my password.
Observation: Generated BDD test cases for: As a user, I want to reset my password using a link sent to my email, so that I can regain access to my account if I forgot my password.
Thought:Final Answer: Feature: Password Reset via Email Link

Scenario: User successfully resets password
  Given I am on the login page
  And I click on "Forgot Password?"
  When I enter my registered email address
  And I click "Send Reset Link"
  Then I should receive an email with a password reset link
  And I click the password reset link in the email
  And I am redirected to the "Reset Password" page
  And I enter a new valid password
  And I confirm the new valid password
  And I click "Reset Password"
  Then I should see a "Password successfully reset" message
  And I should be able to 